## Single track model

A common model to simulate vehicle dynamics is the single track model.
In this notebook, I'm using sympy to set up a simulation of this model.
The derivation can be found here (https://de.wikipedia.org/wiki/Einspurmodell)

TODO: apparently this page is only available in german, but surely there is an english version of this somewhere.

In [1]:
import numpy as np
from sympy import *

In [2]:
from IPython.display import Image
Image(url="https://upload.wikimedia.org/wikipedia/commons/2/24/EinspurKinematik.png")

In [3]:
# parameters of the model
m, theta, l_v, l_h, c_v, c_h, i_S = symbols("m, theta, l_v, l_h, c_v, c_h, i_S")

# some sensible reference values, taken from the wikipedia article
parameters = {
m : 1550,    # kg
theta : 2800, # kg m^2
l_v : 1.344,  # m
l_h : 1.456,  # m
c_v : 75000,  # N / rad
c_h : 150000, # N / rad
i_S : 16
}

In [4]:
# in the single track model, v is assumed to be constant
# here, I'm keeping v symbolic, so that it's possible to plug in different values of v
v = symbols("v")

# state of the model is  x = [beta, psi_dot]
# input of the model is u = [delta / i_S]
beta, psi_dot = symbols("beta psi_dot")
delta = symbols("delta")

In [5]:
# x_dot = A * x + B * u
# here are the corresponding matrices A and B
A = Matrix([[-(c_v + c_h)/(m*v), (m*v**2 - (c_h*l_h - c_v*l_v))/(m*v**2)],
           [- (c_h*l_h - c_v*l_v)/theta, -(c_h*l_h**2 + c_v*l_v**2)/(theta*v)]])

B = Matrix([[-c_v / (m*v)], [c_v*l_v/theta]])

In [6]:
# state space representation: dx = A*x + B*u
def dstate(state, u):
    return A @ state + B@u

In [7]:
# a runge kutta implementation
T = symbols("T")

state = Matrix([beta, psi_dot])
u = Matrix([delta / i_S])

# runge kutta
k1 = dstate(state, u)
k2 = dstate(state + T/2*k1, u)
k3 = dstate(state + T/2*k2, u)
k4 = dstate(state + T*k3, u)

rk = state + T/6 * (k1 + 2*k2 + 2*k3 + k4)

In [8]:
# a sample evaluation of the runge kutta setup

# here we define values to be used for states and inputs
# they will be substituted into the symbolic expression
beta_val = 0
psi_dot_val = 0
delta_val = 0.05
v_val = 10
dT = 1

substitutions = {
    beta: beta_val, 
    psi_dot : psi_dot_val, 
    delta: delta_val, 
    v:v_val,
    T: dT
}

# we also need to substitute the parameters from above
rk.subs({**substitutions, **parameters}).evalf()

Matrix([
[1.89157784341162],
[1.78230636680725]])